In [ ]:
# default_exp modeling.seq2seq.translation

In [ ]:
#all_slow

In [ ]:
#hide
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# modeling.seq2seq.translation

> This module contains custom models, custom splitters, etc... translation tasks.

In [ ]:
#export
import torch
from datasets import list_datasets, load_dataset
from transformers import *
from fastai.text.all import *

from blurr.utils import *
from blurr.data.core import get_blurr_tfm
from blurr.data.seq2seq.core import *
from blurr.data.seq2seq.translation import *
from blurr.modeling.core import *
from blurr.modeling.seq2seq.core import *

logging.set_verbosity_error()

[nltk_data] Downloading package wordnet to /home/wgilliam/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
#hide
import pdb

from nbdev.showdoc import *
from fastcore.test import *

from fastai import __version__ as fa_version
from torch import __version__ as pt_version
from transformers import __version__ as hft_version

print(f'Using pytorch {pt_version}')
print(f'Using fastai {fa_version}')
print(f'Using transformers {hft_version}')

Using pytorch 1.7.1
Using fastai 2.1.8
Using transformers 4.1.1


In [ ]:
#cuda
torch.cuda.set_device(1)
print(f'Using GPU #{torch.cuda.current_device()}: {torch.cuda.get_device_name()}')

Using GPU #1: GeForce GTX 1080 Ti


## Translation

Translation tasks attempt to convert text in one language into another

### Prepare the data

In [ ]:
ds = load_dataset('wmt16', 'de-en', split='train[:1%]')

In [ ]:
path = Path('./')
wmt_df = pd.DataFrame(ds['translation'], columns=['de', 'en']); len(wmt_df)

In [ ]:
wmt_df = wmt_df.iloc[:1000]

In [ ]:
wmt_df.head(2)

In [ ]:
pretrained_model_name = "facebook/bart-large-cnn"
task = HF_TASKS_AUTO.Seq2SeqLM

hf_arch, hf_config, hf_tokenizer, hf_model = BLURR_MODEL_HELPER.get_hf_objects(pretrained_model_name, task=task)

hf_arch, type(hf_tokenizer), type(hf_config), type(hf_model)

In [ ]:
blocks = (HF_Seq2SeqBlock(hf_arch, hf_config, hf_tokenizer, hf_model), noop)
dblock = DataBlock(blocks=blocks, get_x=ColReader('de'), get_y=ColReader('en'), splitter=RandomSplitter())

In [ ]:
dls = dblock.dataloaders(wmt_df, bs=2)

In [ ]:
b = dls.one_batch()

In [ ]:
len(b), b[0]['input_ids'].shape, b[1].shape

In [ ]:
dls.show_batch(dataloaders=dls, max_n=2, input_trunc_at=250, target_trunc_at=250)

### Train model

In [ ]:
seq2seq_metrics = {
    'bleu': { 'returns': "bleu" },
    'meteor': { 'returns': "meteor" },
    'sacrebleu': { 'returns': "score" }
}

model = HF_BaseModelWrapper(hf_model)

learn_cbs = [HF_BaseModelCallback]
fit_cbs = [HF_Seq2SeqMetricsCallback(custom_metrics=seq2seq_metrics)]

learn = Learner(dls, 
                model,
                opt_func=partial(Adam),
                loss_func=CrossEntropyLossFlat(), #HF_PreCalculatedLoss()
                cbs=learn_cbs,
                splitter=partial(seq2seq_splitter, arch=hf_arch)) #.to_native_fp16() #.to_fp16()

learn.create_opt() 
learn.freeze()

In [ ]:
# learn.blurr_summary()

In [ ]:
# b = dls.one_batch()
# preds = learn.model(b[0])

# len(preds),preds['loss'].shape, preds['logits'].shape

In [ ]:
len(b), len(b[0]), b[0]['input_ids'].shape, len(b[1]), b[1].shape

In [ ]:
print(len(learn.opt.param_groups))

In [ ]:
#slow
learn.lr_find(suggestions=True)

In [ ]:
#slow
learn.fit_one_cycle(1, lr_max=4e-5, cbs=fit_cbs)

In [ ]:
learn.show_results(learner=learn, input_trunc_at=500, target_trunc_at=500)

In [ ]:
test_de = "Ich trinke gerne Bier"

In [ ]:
outputs = learn.blurr_generate(test_de, num_return_sequences=3)

for idx, o in enumerate(outputs):
    print(f'=== Prediction {idx+1} ===\n{o}\n')

### Inference

In [ ]:
export_fname = 'translation_export'

In [ ]:
learn.metrics = None
learn.export(fname=f'{export_fname}.pkl')

In [ ]:
inf_learn = load_learner(fname=f'{export_fname}.pkl')
inf_learn.blurr_generate(test_de)

## Tests

The purpose of the following tests is to ensure as much as possible, that the core training code works for the pretrained **summarization models** below.  These tests are excluded from the CI workflow because of how long they would take to run and the amount of data that would be required to download.

**Note**: Feel free to modify the code below to test whatever pretrained summarization models you are working with ... and if any of your pretrained summarization models fail, please submit a github issue *(or a PR if you'd like to fix it yourself)*

In [ ]:
try: del learn; torch.cuda.empty_cache()
except: pass

In [ ]:
BLURR_MODEL_HELPER.get_models(task='ConditionalGeneration')

[transformers.models.bart.modeling_bart.BartForConditionalGeneration,
 transformers.models.blenderbot.modeling_blenderbot.BlenderbotForConditionalGeneration,
 transformers.models.fsmt.modeling_fsmt.FSMTForConditionalGeneration,
 transformers.models.mbart.modeling_mbart.MBartForConditionalGeneration,
 transformers.models.mt5.modeling_mt5.MT5ForConditionalGeneration,
 transformers.models.pegasus.modeling_pegasus.PegasusForConditionalGeneration,
 transformers.models.prophetnet.modeling_prophetnet.ProphetNetForConditionalGeneration,
 transformers.models.t5.modeling_t5.T5ForConditionalGeneration,
 transformers.models.bart.modeling_tf_bart.TFBartForConditionalGeneration,
 transformers.models.blenderbot.modeling_tf_blenderbot.TFBlenderbotForConditionalGeneration,
 transformers.models.mbart.modeling_tf_mbart.TFMBartForConditionalGeneration,
 transformers.models.mt5.modeling_tf_mt5.TFMT5ForConditionalGeneration,
 transformers.models.pegasus.modeling_tf_pegasus.TFPegasusForConditionalGeneration,

In [ ]:
pretrained_model_names = [
    'facebook/bart-base',
    'facebook/wmt19-de-en',                      # FSMT
    'Helsinki-NLP/opus-mt-de-en',                # MarianMT
    'sshleifer/tiny-mbart',
    'google/mt5-small',
    't5-small'
]

In [ ]:
path = Path('./')
ds = load_dataset('wmt16', 'de-en', split='train[:1%]')
wmt_df = pd.DataFrame(ds['translation'], columns=['de', 'en']); len(wmt_df)
wmt_df = wmt_df.iloc[:1000]

Reusing dataset wmt16 (/home/wgilliam/.cache/huggingface/datasets/wmt16/de-en/1.0.0/7b2c4443a7d34c2e13df267eaa8cab4c62dd82f6b62b0d9ecc2e3a673ce17308)


In [ ]:
#slow
#hide_output
task = HF_TASKS_AUTO.Seq2SeqLM
bsz = 2
inp_seq_sz = 128; trg_seq_sz = 128

test_results = []
for model_name in pretrained_model_names:
    error=None
    
    print(f'=== {model_name} ===\n')
    
    hf_arch, hf_config, hf_tokenizer, hf_model = BLURR_MODEL_HELPER.get_hf_objects(model_name, task=task)
    print(f'architecture:\t{hf_arch}\ntokenizer:\t{type(hf_tokenizer).__name__}\nmodel:\t\t{type(hf_model).__name__}\n')
    
    # 1. build your DataBlock
    text_gen_kwargs = default_text_gen_kwargs(hf_config, hf_model, task='translation')
    
    tok_kwargs = {}
    if (hf_arch == 'mbart'):
        tok_kwargs['src_lang'], tok_kwargs['tgt_lang'] = "de_DE", "en_XX"
            
    def add_t5_prefix(inp): return f'translate German to English: {inp}' if (hf_arch == 't5') else inp
    
    before_batch_tfm = HF_Seq2SeqBeforeBatchTransform(hf_arch, hf_config, hf_tokenizer, hf_model,
                                                      padding='max_length', 
                                                      max_length=inp_seq_sz, 
                                                      max_target_length=trg_seq_sz, 
                                                      tok_kwargs=tok_kwargs, text_gen_kwargs=text_gen_kwargs)
    
    blocks = (HF_Seq2SeqBlock(before_batch_tfm=before_batch_tfm), noop)
    dblock = DataBlock(blocks=blocks, 
                   get_x=Pipeline([ColReader('de'), add_t5_prefix]), 
                   get_y=ColReader('en'), 
                   splitter=RandomSplitter())

    dls = dblock.dataloaders(wmt_df, bs=bsz) 
    b = dls.one_batch()

    # 2. build your Learner
    seq2seq_metrics = {}
    
    model = HF_BaseModelWrapper(hf_model)
    fit_cbs = [HF_Seq2SeqMetricsCallback(custom_metrics=seq2seq_metrics)]

    learn = Learner(dls, 
                    model,
                    opt_func=ranger,
                    loss_func=HF_PreCalculatedLoss(),
                    cbs=[HF_BaseModelCallback],
                    splitter=partial(seq2seq_splitter, arch=hf_arch)).to_fp16()

    learn.create_opt() 
    learn.freeze()
    
    # 3. Run your tests
    try:
        print('*** TESTING DataLoaders ***\n')
        test_eq(len(b), 2)
        test_eq(len(b[0]['input_ids']), bsz)
        test_eq(b[0]['input_ids'].shape, torch.Size([bsz, inp_seq_sz]))
        test_eq(len(b[1]), bsz)

#         print('*** TESTING One pass through the model ***')
#         preds = learn.model(b[0])
#         test_eq(preds[1].shape[0], bsz)
#         test_eq(preds[1].shape[2], hf_config.vocab_size)

        print('*** TESTING Training/Results ***')
        learn.fit_one_cycle(1, lr_max=1e-3, cbs=fit_cbs)

        test_results.append((hf_arch, type(hf_tokenizer).__name__, type(hf_model).__name__, 'PASSED', ''))
        learn.show_results(learner=learn, max_n=2, input_trunc_at=500, target_trunc_at=250)
    except Exception as err:
        test_results.append((hf_arch, type(hf_tokenizer).__name__, type(hf_model).__name__, 'FAILED', err))
    finally:
        # cleanup
        del learn; torch.cuda.empty_cache()

=== facebook/bart-base ===

architecture:	bart
tokenizer:	BartTokenizerFast
model:		BartForConditionalGeneration

*** TESTING DataLoaders ***

*** TESTING Training/Results ***


epoch,train_loss,valid_loss,time
0,4.261057,3.973652,01:05


,text,target,prediction
0,"Aus diesem Grund ist es eines der wichtigsten und weitreichendsten Ziele, die wir uns in der Europäischen Union stellen sollten, Anstrengungen zur Schaffung neuer Arbeitsplätze in den ländlichen Gebieten außerhalb des Agrarsektors zu unternehmen, unter anderem in den Bereichen ländlicher Tourismus, Sport, Kultur, Sanierung der Ressourcen,","For this reason, one of the most important and essential objectives which we should set in the European Union is to make efforts to create new jobs in rural areas, outside of the agricultural sector, in sectors such as rural tourism, sport, culture,","In this respect, we should like to thank the rapporteur for his excellent work and"
1,"Vor allen Dingen besteht die Gefahr, daß die - in vielerlei Hinsicht notwendige - Verlagerung der Zuständigkeiten auf die einzelstaatliche Ebene dazu führt, daß Maßnahmen im Bereich des Wettbewerbs enorm zunehmen und die Versuchung besteht, das Kartellverbot nicht als letztinstanzliche Garantie für das einwandfreie und","First of all, there is a risk that the decentralisation of powers, though necessary in many ways, will cause an abnormal increase in competition-related initiatives, and that some people will be tempted to use competition law, not as a means to be r","In all of this respect, it is important to make it clear that, in the"


=== facebook/wmt19-de-en ===

architecture:	fsmt
tokenizer:	FSMTTokenizer
model:		FSMTForConditionalGeneration

*** TESTING DataLoaders ***

*** TESTING Training/Results ***


epoch,train_loss,valid_loss,time
0,1.435724,1.532042,03:54


...

epoch,train_loss,valid_loss,time
0,nan,nan,01:06


,text,target,prediction
0,"Sie wird aber auf Seite 5 dieser Leitlinien ganz eindeutig genannt, und ich möchte darauf verweisen - weil sie mich dazu aufgefordert haben -, daß diese Partnerschaft für mich - und ich habe lange genug eine Region betreut, um dies beurteilen zu können - ein sehr wirkungsvolles Instrument zur Mobilisierung der geistigen Ressourcen auf lokaler Ebene ist - sowohl derer im öffentlichen Sektor - die Stadt- und Gemeinderäte, den schulischen und gesellschaftlichen Bereich,","However, I do wish to mention - since you have asked me to do so - that, as far as I am concerned, this partnership - and I spent long enough as a regional administrator within my own country to be able to say this most sincerely - is a tool, one use",<extra_id_0>.
1,"Ich gehe mit der Berichterstatterin konform, daß das Kommissionsdokument leider zu wenig Empfehlungen an die Mitgliedstaaten zur Verwaltungsvereinfachung enthält, und unterstreiche Forderungen wie Konzentration bei Verhandlungen auf die Förderung eines günstigen Umfeldes für arbeitsintensive kleine und mittelständische Unternehmen, auf eine klare Zielsetzung für alternative Finanzierungsquellen einschließlich Regelungen für Risikokapital und private Finanzierung, auf betriebliche Starthilfen ein","I agree with the rapporteur that unfortunately the Commission document contains too little in the way of recommendations to the Member States on simplifying administration, and I support the calls for negotiations to concentrate on promoting a favour",<extra_id_0> an


=== t5-small ===

architecture:	t5
tokenizer:	T5TokenizerFast
model:		T5ForConditionalGeneration

*** TESTING DataLoaders ***

*** TESTING Training/Results ***


epoch,train_loss,valid_loss,time
0,0.796533,0.738978,00:52


,text,target,prediction
0,"summarize: Angesichts dieser Situation muß aus dem Bericht, den das Parlament annimmt, klar hervorgehen, daß Maßnahmen notwendig sind, die eindeutig auf die Bekämpfung der relativen Armut und der Arbeitslosigkeit gerichtet sind. Maßnahmen wie die für diese Zwecke angemessene Verwendung der Strukturfonds, die häufig unsachgemäß eingesetzt werden, und zwar mit zentralen staatlichen Politiken, die Modernisierung der Bereiche Telekommunikation und Kommunikation, indem man vor allem die am wenigsten","Given this situation, the report approved by Parliament must highlight the need for measures that aim unequivocally to fight relative poverty and unemployment: measures such as the appropriate use of structural funds for these purposes, which are oft",
1,"summarize: Es muß daran erinnert werden, daß die globale Wettbewerbsfähigkeit der Europäischen Union gegenwärtig 81 % des Niveaus der Vereinigten Staaten von Amerika erreicht und daß diese Kennziffer sich nur dann verbessern wird, wenn sich die unserer wettbewerbsfähigen Wirtschaftseinheiten, nämlich der Regionen, verbessert, und das zu einem Zeitpunkt, da die technologische Entwicklung, die Globalisierung der Wirtschaft und unsere Probleme, die Erweiterung und die Einheitswährung, von den Regio","It must be remembered that, currently, the European Union' s overall competitiveness is, in general terms, 81% of that of the United States of America and that this figure will only improve if the figure for our competitive units, that is the regions",


In [ ]:
#slow
#hide_input
test_results_df = pd.DataFrame(test_results, columns=['arch', 'tokenizer', 'model_name', 'result', 'error'])
display_df(test_results_df)

,arch,tokenizer,model_name,result,error
0,bart,BartTokenizerFast,BartForConditionalGeneration,PASSED,
1,fsmt,FSMTTokenizer,FSMTForConditionalGeneration,PASSED,
2,marian,MarianTokenizer,MarianMTModel,PASSED,
3,mbart,MBartTokenizerFast,MBartForConditionalGeneration,PASSED,
4,mt5,T5TokenizerFast,MT5ForConditionalGeneration,PASSED,
5,t5,T5TokenizerFast,T5ForConditionalGeneration,PASSED,


## Cleanup

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_utils.ipynb.
Converted 01_data-core.ipynb.
Converted 01a_data-token-classification.ipynb.
Converted 01b_data-question-answering.ipynb.
Converted 01za_data-seq2seq-core.ipynb.
Converted 01zb_data-seq2seq-language-modeling.ipynb.
Converted 01zc_data-seq2seq-summarization.ipynb.
Converted 01zd_data-seq2seq-translation.ipynb.
Converted 02_modeling-core.ipynb.
Converted 02a_modeling-token-classification.ipynb.
Converted 02b_modeling-question-answering.ipynb.
Converted 02za_modeling-seq2seq-core.ipynb.
Converted 02zb_modeling-seq2seq-language-modeling.ipynb.
Converted 02zc_modeling-seq2seq-summarization.ipynb.
Converted 02zc_modeling-seq2seq-translation.ipynb.
Converted 99a_examples-multilabel.ipynb.
Converted index.ipynb.
